In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import random

# === Set seed for full reproducibility ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# === Load and prepare dataset ===
df = pd.read_csv(r"resources/data.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["FoS"].values  # 🔄 Change to: df["SeismicFoS"].values for SeismicFoS

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === Define ANN model ===
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1),
            nn.ReLU(),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Linear(h2, h3),
            nn.ReLU(),
            nn.Linear(h3, 1)
        )

    def forward(self, x):
        return self.model(x)

# === Evaluation function (reproducible) ===
def evaluate_model(h1, h2, h3, lr):
    set_seed(42)
    model = ANN(X_train.shape[1], h1, h2, h3)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_test_tensor).numpy()
        score = r2_score(y_test, preds)
    return score

# === GA operations ===
def generate_individual():
    return [
        random.randint(16, 128),
        random.randint(16, 128),
        random.randint(16, 128),
        round(random.uniform(0.0001, 0.01), 5)
    ]

def crossover(p1, p2):
    idx = random.randint(1, 3)
    return p1[:idx] + p2[idx:]

def mutate(ind):
    idx = random.randint(0, 3)
    if idx < 3:
        ind[idx] = random.randint(16, 128)
    else:
        ind[idx] = round(random.uniform(0.0001, 0.01), 5)
    return ind

# === GA Hyperparameters ===
POP_SIZE = 10
GENERATIONS = 10
MUTATION_RATE = 0.3

# === Run Genetic Algorithm ===
population = [generate_individual() for _ in range(POP_SIZE)]
global_best = None
global_best_score = -np.inf

for gen in range(GENERATIONS):
    print(f"\n🔄 Generation {gen+1}")
    scores = []
    for ind in population:
        score = evaluate_model(*ind)
        scores.append(score)
        if score > global_best_score:
            global_best = ind
            global_best_score = score

    sorted_pop = [x for _, x in sorted(zip(scores, population), reverse=True)]
    print("Top individual:", sorted_pop[0], "| R² =", max(scores))

    # Selection and reproduction
    next_gen = sorted_pop[:POP_SIZE // 2]
    while len(next_gen) < POP_SIZE:
        p1, p2 = random.sample(next_gen, 2)
        child = crossover(p1, p2)
        if random.random() < MUTATION_RATE:
            child = mutate(child)
        next_gen.append(child)
    population = next_gen

# === Final Best Result ===
print("\n🏆 Best Overall Individual:", global_best)
print("✅ Best R² (Test):", global_best_score)



🔄 Generation 1
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 2
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 3
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 4
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 5
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 6
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 7
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 8
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 9
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🔄 Generation 10
Top individual: [36, 16, 53, 0.00787] | R² = 0.9172854388325687

🏆 Best Overall Individual: [36, 16, 53, 0.00787]
✅ Best R² (Test): 0.9172854388325687


In [ ]:
#Below code take these hyperparameters as structure for GA-ANN

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# === Set seed for reproducibility ===
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# === Load and prepare dataset ===
df = pd.read_csv(r"resources/data.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["FoS"].values  # change to "SeismicFoS" if needed

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === ANN definition ===
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )
    def forward(self, x):
        return self.model(x)

# === Train and evaluate ===
def train_and_evaluate(h1, h2, h3, lr):
    set_seed()
    model = ANN(X_train.shape[1], h1, h2, h3)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor).numpy()
        y_test_pred = model(X_test_tensor).numpy()

    results = {
        "R2_Train": r2_score(y_train, y_train_pred),
        "R2_Test": r2_score(y_test, y_test_pred),
        "RMSE_Train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "RMSE_Test": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "MAE_Train": mean_absolute_error(y_train, y_train_pred),
        "MAE_Test": mean_absolute_error(y_test, y_test_pred),
    }
    return results

# === Run with given hyperparameters ===
h1, h2, h3, lr = 83, 33, 92, 0.00331
results = train_and_evaluate(h1, h2, h3, lr)

# === Print results ===
print("\n📊 Best GA-ANN Evaluation with Hyperparameters [83, 33, 92, 0.00331]")
for metric, value in results.items():
    print(f"{metric}: {value:.6f}")



📊 Best GA-ANN Evaluation with Hyperparameters [83, 33, 92, 0.00331]
R2_Train: 0.993754
R2_Test: 0.936205
RMSE_Train: 0.091130
RMSE_Test: 0.321321
MAE_Train: 0.061539
MAE_Test: 0.214330


In [ ]:
#SeismicFoS

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import random

# === Set seed for full reproducibility ===
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# === Load and prepare dataset ===
df = pd.read_csv(r"resources/data.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["SeismicFoS"].values  # 🔄 Change to: df["SeismicFoS"].values for SeismicFoS

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === Define ANN model ===
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1),
            nn.ReLU(),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Linear(h2, h3),
            nn.ReLU(),
            nn.Linear(h3, 1)
        )

    def forward(self, x):
        return self.model(x)

# === Evaluation function (reproducible) ===
def evaluate_model(h1, h2, h3, lr):
    set_seed(42)
    model = ANN(X_train.shape[1], h1, h2, h3)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = model(X_test_tensor).numpy()
        score = r2_score(y_test, preds)
    return score

# === GA operations ===
def generate_individual():
    return [
        random.randint(16, 128),
        random.randint(16, 128),
        random.randint(16, 128),
        round(random.uniform(0.0001, 0.01), 5)
    ]

def crossover(p1, p2):
    idx = random.randint(1, 3)
    return p1[:idx] + p2[idx:]

def mutate(ind):
    idx = random.randint(0, 3)
    if idx < 3:
        ind[idx] = random.randint(16, 128)
    else:
        ind[idx] = round(random.uniform(0.0001, 0.01), 5)
    return ind

# === GA Hyperparameters ===
POP_SIZE = 10
GENERATIONS = 10
MUTATION_RATE = 0.3

# === Run Genetic Algorithm ===
population = [generate_individual() for _ in range(POP_SIZE)]
global_best = None
global_best_score = -np.inf

for gen in range(GENERATIONS):
    print(f"\n🔄 Generation {gen+1}")
    scores = []
    for ind in population:
        score = evaluate_model(*ind)
        scores.append(score)
        if score > global_best_score:
            global_best = ind
            global_best_score = score

    sorted_pop = [x for _, x in sorted(zip(scores, population), reverse=True)]
    print("Top individual:", sorted_pop[0], "| R² =", max(scores))

    # Selection and reproduction
    next_gen = sorted_pop[:POP_SIZE // 2]
    while len(next_gen) < POP_SIZE:
        p1, p2 = random.sample(next_gen, 2)
        child = crossover(p1, p2)
        if random.random() < MUTATION_RATE:
            child = mutate(child)
        next_gen.append(child)
    population = next_gen

# === Final Best Result ===
print("\n🏆 Best Overall Individual:", global_best)
print("✅ Best R² (Test):", global_best_score)



🔄 Generation 1
Top individual: [62, 89, 40, 0.00708] | R² = 0.8305807564246461

🔄 Generation 2
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 3
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 4
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 5
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 6
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 7
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 8
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 9
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🔄 Generation 10
Top individual: [62, 33, 40, 0.00887] | R² = 0.8431348275396757

🏆 Best Overall Individual: [62, 33, 40, 0.00887]
✅ Best R² (Test): 0.8431348275396757


In [ ]:
#SeismicFos ANN model based best obtained hyperparameters through GA and these are [62, 33, 40, 0.00887]
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# === Set seed for reproducibility ===
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# === Load and prepare dataset ===
df = pd.read_csv(r"resources/data.csv")
X = df.drop(columns=["FoS", "SeismicFoS"]).values
y = df["SeismicFoS"].values  # Use SeismicFoS

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# === Define ANN model ===
class ANN(nn.Module):
    def __init__(self, input_dim, h1, h2, h3):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, h3), nn.ReLU(),
            nn.Linear(h3, 1)
        )

    def forward(self, x):
        return self.model(x)

# === Train and evaluate function ===
def train_and_evaluate(h1, h2, h3, lr):
    set_seed()
    model = ANN(X_train.shape[1], h1, h2, h3)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(100):
        model.train()
        for xb, yb in train_loader:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        y_train_pred = model(X_train_tensor).numpy()
        y_test_pred = model(X_test_tensor).numpy()

    results = {
        "R2_Train": r2_score(y_train, y_train_pred),
        "R2_Test": r2_score(y_test, y_test_pred),
        "RMSE_Train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "RMSE_Test": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "MAE_Train": mean_absolute_error(y_train, y_train_pred),
        "MAE_Test": mean_absolute_error(y_test, y_test_pred),
    }
    return results

# === Evaluate with provided hyperparameters ===
h1, h2, h3, lr = 62, 33, 40, 0.00887
results = train_and_evaluate(h1, h2, h3, lr)

# === Print results ===
print("\n📊 ANN Evaluation for SeismicFoS with Hyperparameters [62, 33, 40, 0.00887]")
for metric, value in results.items():
    print(f"{metric}: {value:.6f}")



📊 ANN Evaluation for SeismicFoS with Hyperparameters [62, 33, 40, 0.00887]
R2_Train: 0.993777
R2_Test: 0.843135
RMSE_Train: 0.068895
RMSE_Test: 0.371873
MAE_Train: 0.050461
MAE_Test: 0.241317
